In [1]:
import numpy as np
import gym
# https://ray.readthedocs.io/en/latest/rllib-env.html
from ray.rllib.agents import ppo
import time
import ray
from ray import tune
import or_gym
import matplotlib.pyplot as plt
from or_gym.algos.rl_utils import *
%matplotlib inline
plt.style.use('cmu_paper')

In [2]:
ray.init(ignore_reinit_error=True)
env_name = 'VehicleRouting-v1'
trainer = ppo.PPOTrainer(env=create_env(env_name), 
    config={
    "env_config": {
        "reuse_actors":True, 
        "num_vehicles": 1},
    "vf_clip_param": 500,
    "model": {
        "fcnet_activation": "elu",
        "fcnet_hiddens": [128, 128, 128]}
    })

# Run x number of episodes
N_EPS = 1000
rewards = []
eps, eps_total = [], []
training = True
batch = 0
t_start = time.time()
while training:
    t_batch = time.time()
    results = trainer.train()
    rewards.append(results['episode_reward_mean'])
    eps.append(results['episodes_this_iter'])
    eps_total.append(results['episodes_total'])
    batch += 1
    t_end = time.time()
    if sum(eps) >= N_EPS:
        training = False
        break
    if batch % 10 == 0:
        t = t_end - t_batch
        t_tot = t_end - t_start
        print("\rEpisode: {}\tMean Rewards: {:.1f}\tEpisodes/sec: {:.2f}s\tTotal Time: {:.1f}s".format(
            eps_total[-1], rewards[-1], eps[-1]/t, t_tot), end="")
        
print("Total Training Time: {:.1f}s\t".format(t_end - t_start))

2020-04-02 17:33:28,914	WARNING services.py:597 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-04-02 17:33:28,917	INFO resource_spec.py:216 -- Starting Ray with 2.78 GiB memory available for workers and up to 1.39 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-02 17:33:29,431	INFO trainer.py:371 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-04-02 17:33:29,442	INFO trainer.py:512 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


Environment	VehicleRouting-v1


2020-04-02 17:33:49,358	INFO trainable.py:102 -- _setup took 19.917 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2020-04-02 17:33:49,359	WARNING util.py:45 -- Install gputil for GPU system monitoring.


RayTaskError(ValueError): [36mray::RolloutWorker.sample()[39m (pid=22479, ip=192.168.0.11)
  File "python/ray/_raylet.pyx", line 636, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 619, in ray._raylet.execute_task.function_executor
  File "/home/christian/anaconda3/lib/python3.6/site-packages/ray/rllib/evaluation/rollout_worker.py", line 471, in sample
    batches = [self.input_reader.next()]
  File "/home/christian/anaconda3/lib/python3.6/site-packages/ray/rllib/evaluation/sampler.py", line 56, in next
    batches = [self.get_data()]
  File "/home/christian/anaconda3/lib/python3.6/site-packages/ray/rllib/evaluation/sampler.py", line 99, in get_data
    item = next(self.rollout_provider)
  File "/home/christian/anaconda3/lib/python3.6/site-packages/ray/rllib/evaluation/sampler.py", line 340, in _env_runner
    base_env.send_actions(actions_to_send)
  File "/home/christian/anaconda3/lib/python3.6/site-packages/ray/rllib/env/base_env.py", line 332, in send_actions
  

In [2]:
env = gym.make('VehicleRouting-v1')
env.action_space.sample()

(1, 4)

In [4]:
results

{'episode_reward_max': -50.97048045790419,
 'episode_reward_min': -219.18840948538084,
 'episode_reward_mean': -119.84630218875076,
 'episode_len_mean': 32.0,
 'episodes_this_iter': 126,
 'policy_reward_min': {},
 'policy_reward_max': {},
 'policy_reward_mean': {},
 'custom_metrics': {},
 'sampler_perf': {'mean_env_wait_ms': 0.1993907868031464,
  'mean_processing_ms': 0.1705116826618815,
  'mean_inference_ms': 0.6982471397404194},
 'off_policy_estimator': {},
 'info': {'num_steps_trained': 31744,
  'num_steps_sampled': 32000,
  'sample_time_ms': 2268.965,
  'load_time_ms': 9.827,
  'grad_time_ms': 3133.149,
  'update_time_ms': 97.104,
  'learner': {'default_policy': {'cur_kl_coeff': 0.675000011920929,
    'cur_lr': 4.999999873689376e-05,
    'total_loss': 59.74801,
    'policy_loss': -0.056361508,
    'vf_loss': 59.79441,
    'vf_explained_var': 0.91408396,
    'kl': 0.01476403,
    'entropy': 1.4511726,
    'entropy_coeff': 0.0}}},
 'timesteps_this_iter': 4000,
 'done': False,
 'times

In [3]:
from collections import Iterable
from gym import spaces

In [6]:
tuple([spaces.Discrete(2) for i in range(3)])

(Discrete(2), Discrete(2), Discrete(2))

In [7]:
env = gym.make('VehicleRouting-v1')
env.reset()

for ep in range(1000):
    done = False
    rewards = 0
    while done == False:
        action = env.action_space.sample()
        s, r, done, _ = env.step(action)
        rewards += r
        if done and ep % 100==0:
            print('Ep {}\tReward{}'.format(ep, rewards))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [6]:
env.action_space.sample()

(0, 3)

In [9]:
env.spec.id

'VehicleRouting-v1'

In [24]:
dir(or_gym.classic_or)

['BinPackingEnv',
 'BoundedKnapsackEnv',
 'KnapsackEnv',
 'MultiLevelNewsVendorEnv',
 'OnlineKnapsackEnv',
 'TempVMPackingEnv',
 'VMPackingEnv',
 'VehicleRoutingEnv',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'binpacking',
 'knapsack',
 'newsvendor',
 'vehicle_routing',
 'vmpacking']

In [25]:
import string

In [26]:
string.ascii_uppercase

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [40]:
x = gym.envs

In [46]:
x.registry.all

['/home/christian/anaconda3/lib/python3.6/site-packages/gym']

In [3]:
or_gym.envs.registry.registry

AttributeError: 'EnvRegistry' object has no attribute 'registry'

In [47]:
or_gym.envs

<function gym.envs.registration.register(id, **kwargs)>

In [4]:
gym.make('Knapsack-v0')

UnregisteredEnv: No registered env with id: Knapsack-v0